In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import os
import pandas as pd
import copy
from PIL import Image
%matplotlib inline

In [6]:
tr = pd.read_csv('./sun397_train_lt.txt', header=None, sep=' ')

In [7]:
class_names = {}

In [8]:
for lab in tr[1].unique():
    temp = tr.loc[tr[1] == lab].iloc[0, 0]
    class_name = temp.split('/')[-2]
    class_names[lab] = class_name

In [9]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [10]:
#  device = torch.device('cpu')

In [11]:
class sun_dataset (torch.utils.data.Dataset):
    
    def __init__ (self, txt_file, transform=None):
        super().__init__()
        self.df = pd.read_csv(txt_file, header=None, sep=' ')
        self.transform = transform
        print('Loading from %s' % txt_file)
        
    def __len__ (self):
        return len(self.df)
    
    def __getitem__ (self, idx):
        
        image = Image.open(self.df.iloc[idx, 0])
        label = self.df.iloc[idx, 1] - 1
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

def train_model (model, dataloaders, dataset_sizes, batch_size, num_classes, loss_function, 
                 optimizer, scheduler, num_epochs, device, display_step, log_dir, model_id=None):
    
    # Deep copy model weights
    best_model_weights = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    # Start training
    training_step = 0
    for epoch in range(num_epochs):
          
        # Loop over training phase and validation phase
        for phase in ['train', 'val']:
            
            # Set model modes and set scheduler
            if phase == 'train':
                scheduler.step()
                model.train()
            else:
                model.eval()
                
            running_loss = 0.0
            running_correct_total = 0
            
            class_correct = torch.tensor([0. for i in range(num_classes)])
            class_total = torch.tensor([0. for i in range(num_classes)])
            
            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                
                inputs, labels = inputs.to(device), labels.to(device)
                
                # Zero parameter gradients
                optimizer.zero_grad()
                
                # Forward
                # If on training phase, enable gradients
                with torch.set_grad_enabled(phase == 'train'):
                    
                    logits = model(inputs)
                    _, preds = torch.max(logits, 1)
                    loss = loss_function(logits, labels)
                    
                    # Backward if training
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        training_step += 1
                        
                        if training_step % display_step == 0:
                            minibatch_loss = loss.item()
                            minibatch_acc = (preds == labels).sum().item() / batch_size
                            print('Epoch: %d, Step: %5d, Minibatch_loss: %.3f, Minibatch_accuracy_micro: %.3f' 
                                  % (epoch, training_step, minibatch_loss, minibatch_acc))
                        
                # Record loss and correct predictions
                correct_tensor = (preds == labels).squeeze()
                running_loss += loss.item() * inputs.shape[0]
                running_correct_total += correct_tensor.sum().item()
                
                for i in range(len(labels)):
                    label = labels[i]
                    class_correct[label] += correct_tensor[i].item()
                    class_total[label] += 1
                
            # Epoch loss and accuracieds
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc_mic = running_correct_total / dataset_sizes[phase]
            epoch_acc_mac = (class_correct / class_total).mean().item()
            
            print('Epoch: %d, Phase: %s, Epoch_loss: %.3f, Epoch_accuracy_micro: %.3f, Epoch_accuracy_macro: %.3f' 
                  % (epoch, phase, epoch_loss, epoch_acc_mic, epoch_acc_mac))
            
            # Deep copy the best model weights
            if phase == 'val' and epoch_acc_mic > best_acc:
                best_acc = epoch_acc_mic
                best_model_weights = copy.deepcopy(model.state_dict())
                
    print()
    print('Training Complete.')
    print('Best validation accuracy: %.3f' % best_acc)
    
    # Load the best model weights
    model.load_state_dict(best_model_weights)
    
    # Save the best model
    model_states = {'epoch': epoch + 1,
                    'state_dict': model.state_dict(),
                    'best_acc': best_acc,
                    'optimizer' : optimizer.state_dict()}
    
    torch.save(model_states, os.path.join(log_dir, 'model_%s_checkpoint.pth.tar' % model_id))
    
    return model

class MyResNet (nn.Module):
    
    def __init__ (self, pretrained_resnet, num_classes, drop_rate):
        super().__init__()
        self.pretrained_resnet = pretrained_resnet
        
        # Reset the fc layer
        self.num_features = self.pretrained_resnet.fc.in_features
        
        self.fc_add = nn.Linear(self.num_features, 4096)
        self.dropout = nn.Dropout(p=drop_rate)
        self.fc = nn.Linear(4096, num_classes)
        
    def forward(self, x):
        x = self.pretrained_resnet.conv1(x)
        x = self.pretrained_resnet.bn1(x)
        x = self.pretrained_resnet.relu(x)
        x = self.pretrained_resnet.maxpool(x)

        x = self.pretrained_resnet.layer1(x)
        x = self.pretrained_resnet.layer2(x)
        x = self.pretrained_resnet.layer3(x)
        x = self.pretrained_resnet.layer4(x)

        x = self.pretrained_resnet.avgpool(x)
        x = x.view(x.size(0), -1)
        
        x = self.fc_add(x)
        x = self.dropout(x)
        x = self.fc(x)

        return x

In [16]:
LOG_DIR = './log'
DATASET = 'plain'
MODEL_ID = 'test_dropout'
DATALOADER_WORKERS = 4
LEARNING_RATE = 0.01
LR_DECAY_FACTOR = 0.1
LR_DECAY_EPOCHS = 10
DROPOUT = True
DROPOUT_RATE = 0.5
MOMENTUM = 0.9
EPOCHS = 1
BATCH_SIZE = 256
DISPLAY_STEP = 10
NUM_CLASSES = 397

if not os.path.isdir(LOG_DIR):
    os.makedirs(LOG_DIR)

In [17]:
# transforms.RandomResizeCrop(224)
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

datasets = {x: sun_dataset(txt_file='./sun397_%s_lt.txt' % x, transform=data_transforms[x]) for x in ['train', 'val']} 
dataloaders = {x: torch.utils.data.DataLoader(datasets[x], batch_size=BATCH_SIZE, shuffle=True, num_workers=DATALOADER_WORKERS) for x in ['train', 'val']}
dataset_sizes = {x: len(datasets[x]) for x in ['train', 'val']}

Loading from ./sun397_train_lt.txt
Loading from ./sun397_val_lt.txt


In [18]:
# Load pretrained model
resnet = torchvision.models.resnet152(pretrained=True)
# Freeze all layers
for param in resnet.parameters():
    param.requires_grad = False

In [19]:
if DROPOUT:
    print('Model using dropout.')
    resnet = MyResNet(resnet, NUM_CLASSES, DROPOUT_RATE)
    # Optimizer only on the last fc layers
    optimizer = optim.SGD(list(resnet.fc.parameters()) + list(resnet.fc_add.parameters()), lr=LEARNING_RATE, momentum=MOMENTUM)
else:
    print('Model not using dropout.')
    # Reset the fc layer
    num_features = resnet.fc.in_features
    resnet.fc = nn.Linear(num_features, NUM_CLASSES)
    # Optimizer only on the last fc layers
    optimizer = optim.SGD(list(resnet.fc.parameters()), lr=LEARNING_RATE, momentum=MOMENTUM)

Model using dropout.


In [20]:
resnet = resnet.to(device)

# Loss function
loss_function = nn.CrossEntropyLoss()

# Decay LR by a factor of 0.1 every 30 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

In [22]:
# resnet = train_model(model=resnet, loss_function=loss_function, optimizer=optimizer, scheduler=exp_lr_scheduler, num_epochs=EPOCHS, model_id='plain')

In [23]:
resnet = train_model(model=resnet, dataloaders=dataloaders, batch_size=BATCH_SIZE,
                     dataset_sizes=dataset_sizes, num_classes=NUM_CLASSES, loss_function=loss_function, 
                     optimizer=optimizer, scheduler=exp_lr_scheduler, 
                     device=device, num_epochs=EPOCHS, display_step=DISPLAY_STEP,
                     log_dir=LOG_DIR, model_id=MODEL_ID)

Epoch: 0, Step:    10, Minibatch_loss: 5.357, Minibatch_accuracy_micro: 0.035
Epoch: 0, Step:    20, Minibatch_loss: 4.811, Minibatch_accuracy_micro: 0.141
Epoch: 0, Step:    30, Minibatch_loss: 4.388, Minibatch_accuracy_micro: 0.238
Epoch: 0, Step:    40, Minibatch_loss: 4.291, Minibatch_accuracy_micro: 0.234
Epoch: 0, Step:    50, Minibatch_loss: 3.760, Minibatch_accuracy_micro: 0.336
Epoch: 0, Step:    60, Minibatch_loss: 3.765, Minibatch_accuracy_micro: 0.324
Epoch: 0, Step:    70, Minibatch_loss: 3.594, Minibatch_accuracy_micro: 0.320
Epoch: 0, Step:    80, Minibatch_loss: 3.282, Minibatch_accuracy_micro: 0.340
Epoch: 0, Step:    90, Minibatch_loss: 3.135, Minibatch_accuracy_micro: 0.402
Epoch: 0, Step:   100, Minibatch_loss: 3.098, Minibatch_accuracy_micro: 0.398
Epoch: 0, Step:   110, Minibatch_loss: 2.939, Minibatch_accuracy_micro: 0.461
Epoch: 0, Step:   120, Minibatch_loss: 2.859, Minibatch_accuracy_micro: 0.418
Epoch: 0, Step:   130, Minibatch_loss: 2.870, Minibatch_accuracy

# Dropout test

In [3]:
class test_net (nn.Module):
    
    def __init__ (self):
        super().__init__()
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.dropout(x)
        return x

In [9]:
net = test_net()

In [10]:
x = torch.randn(5, 2)

In [21]:
x

tensor([[ 0.3013, -0.4943],
        [ 1.2686,  0.4310],
        [-0.3772,  2.0747],
        [-0.6059, -0.7574],
        [ 1.3811,  0.0971]])

In [28]:
net.eval()

test_net(
  (dropout): Dropout(p=0.5)
)

In [30]:
net(x)

tensor([[ 0.3013, -0.4943],
        [ 1.2686,  0.4310],
        [-0.3772,  2.0747],
        [-0.6059, -0.7574],
        [ 1.3811,  0.0971]])

# Augmented dataset refinement

In [57]:
aug = pd.read_csv('./sun397_train_lt_with_aug.txt', header=None, sep=' ')

num = []

for l in aug[1].unique():
    num.append(len(aug.loc[aug[1] == l]))

fail = []

aug.index

aug.iloc[0, 0]

for ind in range(len(aug)):
    
    d = aug.iloc[ind, 0]
    
    if not os.path.isfile(d):
        d = d.rsplit('aug', 1)
        d = d[0] + '/aug' + d[1]
        assert(os.path.isfile(d))
        aug.iloc[ind, 0] = d

aug.to_csv('./sun397_train_lt_with_aug_new.txt', sep=' ', header=None, index=None)

aug = pd.read_csv('./sun397_train_lt_with_aug_new.txt', header=None, sep=' ')

for d in aug[0]:
    if not os.path.isfile(d):
        print(d)